In [1]:
import sklearn.datasets as datasets
import pandas as pd

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
#bring in the six packs
train = pd.read_csv('data/housing_price_train.csv')
test = pd.read_csv('data/housing_price_test.csv')
#Save the 'Id' column
train_ID = train['Id']
test_ID = test['Id']

#Now drop the  'Id' colum since it's unnecessary for  the prediction process.
train.drop("Id", axis = 1, inplace = True)
test.drop("Id", axis = 1, inplace = True)
ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train.SalePrice.values
all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.drop(['SalePrice'], axis=1, inplace=True)
print("all_data size is : {}".format(all_data.shape))

all_data size is : (3020, 79)


In [3]:
#Check remaining missing values if any 
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head()

,Missing Ratio
PoolQC,99.668874
MiscFeature,96.390728
Alley,93.112583
Fence,80.331126
FireplaceQu,48.675497


In [4]:
all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))

In [5]:
all_data["PoolQC"] = all_data["PoolQC"].fillna("None")

In [6]:
all_data["MiscFeature"] = all_data["MiscFeature"].fillna("None")

In [7]:
all_data["Alley"] = all_data["Alley"].fillna("None")

In [8]:
all_data["Fence"] = all_data["Fence"].fillna("None")

In [9]:
all_data["FireplaceQu"] = all_data["FireplaceQu"].fillna("None")

In [10]:
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    all_data[col] = all_data[col].fillna('None')

In [11]:
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    all_data[col] = all_data[col].fillna(0)

In [12]:
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    all_data[col] = all_data[col].fillna(0)

In [13]:
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    all_data[col] = all_data[col].fillna('None')

In [14]:
all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)
all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])
all_data = all_data.drop(['Utilities'], axis=1)
all_data["Functional"] = all_data["Functional"].fillna("Typ")
all_data['Electrical'] = all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])
all_data['KitchenQual'] = all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode()[0])
all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])
all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode()[0])
all_data['MSSubClass'] = all_data['MSSubClass'].fillna("None")

In [15]:
#Check remaining missing values if any 
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head()

,Missing Ratio


In [16]:
#MSSubClass=The building class
all_data['MSSubClass'] = all_data['MSSubClass'].apply(str)


#Changing OverallCond into a categorical variable
all_data['OverallCond'] = all_data['OverallCond'].astype(str)


#Year and month sold are transformed into categorical features.
all_data['YrSold'] = all_data['YrSold'].astype(str)
all_data['MoSold'] = all_data['MoSold'].astype(str)
from sklearn.preprocessing import LabelEncoder
cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'BldgType', 'MoSold', 'SaleType', 'Condition1', 'Condition2', 'Electrical', 
        'Exterior1st', 'Exterior2nd', 'Foundation', 'GarageType')
# process columns, apply LabelEncoder to categorical features
for c in all_data.columns.values:
    if all_data[c].dtype != np.number:
    #for c in cols:
        lbl = LabelEncoder() 
        lbl.fit(list(all_data[c].values)) 
        all_data[c] = lbl.transform(list(all_data[c].values))

# shape        
print('Shape all_data: {}'.format(all_data.shape))
# Adding total sqfootage feature 
all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']

Shape all_data: (3020, 78)


In [17]:
x_train = all_data[:ntrain]
x_test = all_data[ntrain:]

In [19]:
from sklearn.ensemble import RandomForestClassifier
rforest_hprice = RandomForestClassifier(criterion='entropy', n_estimators=100, random_state=1)
rforest_hprice.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=1, verbose=0, warm_start=False)

In [20]:
y_pred_rf = rforest_hprice.predict(x_test)
y_train_pred = rforest_hprice.predict(x_train)
y_pred = rforest_hprice.predict(x_test)

In [30]:
from sklearn.tree import DecisionTreeClassifier
dtree_hprice = DecisionTreeClassifier(criterion='entropy')
dtree_hprice.fit(x_train, y_train)
y_pred = dtree_hprice.predict(x_test)

In [31]:
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(np.log(y_train), np.log(y_train_pred)))

0.0

In [32]:
y_train, y_train_pred

(array([208500, 181500, 223500, ..., 266500, 142125, 147500]),
 array([208500, 181500, 223500, ..., 266500, 142125, 147500]))

In [33]:
def createSubmission(test_ids, predictions):
    sub = pd.DataFrame()
    sub['Id'] = test_ids
    sub['SalePrice'] = predictions
    sub.to_csv('submission.csv',index=False)

In [34]:
createSubmission(test_ID, y_pred)

In [35]:
gtfile="../../tests/assignment2/groundtruth.csv"

In [36]:
gt = pd.read_csv(gtfile)

In [37]:
gt_values = gt['SalePrice']

In [38]:
score = 1 - (np.sqrt(mean_squared_error(np.log1p(y_pred), np.log1p(gt_values.values))))

In [40]:
score*100

79.3149802036501